# Configuration and imports

In [17]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from dataclasses import dataclass
import matplotlib.pyplot as plt
import plotly.graph_objects as go

@dataclass
class Configuration:
    DATA_FOLDER: str = "data"
    raw_path: str = f"{DATA_FOLDER}/raw"
    processed_path: str = f"{DATA_FOLDER}/processed"


CONFIG = Configuration()


# Loading experiments

In [19]:
from maikol_utils.file_utils import list_dir_files # Mi librería personal :p
import re


class Experiment:
    def __init__(self, path: str):
        self.path = path

        # Las clases de LC patrocinan este regex ;)
        # "GA_Log_20251016_233329-Aleo_Simple-Pop_20000-Mut_0.15-Pieces_10-Seed_42.log",
        pattern = r"GA_Log_(\d{8}_\d{6})-Aleo_([\w-]+)-Pop_(\d+)-Mut_(\d+\.?\d*)-Pieces_(\d+)-Seed_(\d+)"
        match = re.search(pattern, path)
        if match:
            self.date = match.group(1)
            self.agent = match.group(2)
            self.population = int(match.group(3))
            self.mutation_rate = float(match.group(4))
            self.pieces = int(match.group(5))
            self.seed = int(match.group(6))
        else:
            raise ValueError(f"Invalid experiment folder name: {path}")
        
    def __repr__(self):
        # return (f"Experiment(date={self.date}, agent={self.agent}, population={self.population}, "
        #         f"mutation_rate={self.mutation_rate}, pieces={self.pieces}, seed={self.seed})")
        return (f"GEN_{self.agent}-POP_{self.population}-MUT_{self.mutation_rate:.0%}-NP_{self.pieces}")

    def load_log(self):
        gens, fits = [], []
        with open(self.path, "r") as f:
            for line in f:
                if "Generation" in line:
                    parts = line.strip().split(". ")
                    generation = int(parts[0].split(": ")[1])
                    best_fitness = float(parts[1].split(": ")[1])
                    gens.append(generation)
                    fits.append(best_fitness)
        return gens, fits




In [20]:
experiments, n = list_dir_files(CONFIG.raw_path)
print(f"Found {n} experiments in {CONFIG.raw_path}:")

exp_list = []
for exp in experiments:
    exp_list.append(Experiment(exp))

Found 20 experiments in data/raw:


In [21]:
exp

'data/raw/GA_Log_20251017_000128-Aleo_Simple-Pop_30000-Mut_0.05-Pieces_20-Seed_42.log'

In [22]:
def plot_experiments(exp_list):
    fig = go.Figure()

    for exp in exp_list:
        gens, fits = exp.load_log()
        fig.add_trace(go.Scatter(
            x=gens,
            y=fits,
            mode='lines',
            name=repr(exp),
            hovertemplate='Generation: %{x}<br>Best Fitness: %{y:.2f}<extra></extra>'
        ))

    fig.update_layout(
        title="Generation vs Best Fitness - All Experiments",
        xaxis_title="Generation",
        yaxis_title="Best Fitness",
        hovermode='closest',
        width=1200,
        height=700,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=1.01
        )
    )

    fig.show()


# Por Genotipo

In [23]:

plot_experiments(exp_list)